# Example 1: Extract variable

[Extract variable in the refactoring catalog](http://refactoring.com/catalog/extractVariable.html)

---
Some logic to determine the best time to eat certain foods.

In [8]:
MONTHS = ('January', 'February', 'March', 'April', 'May', 'June',
          'July', 'August', 'September', 'October', 'November', 'December')

In [13]:
import random
choice = random.choice(MONTHS)

if (choice.lower().endswith('r') or
        choice.lower().endswith('ary')):
    print('%s is a good time to eat oysters' % choice)
elif 8 > MONTHS.index(choice) > 4:
    print('%s is a good time to eat tomatoes' % choice)
else:
    print('%s is a good time to eat asparagus' % choice)

February is a good time to eat oysters


---

Python creates temporaries for every expression, so there's no cost in extracting variables for the sake of clarity.

In [14]:
lowered = choice.lower()
ends_in_r = lowered.endswith('r')
ends_in_ary = lowered.endswith('ary')
summer = 8 > MONTHS.index(choice) > 4

if ends_in_r or ends_in_ary:
    print('%s is a good time to eat oysters' % choice)
elif summer:
    print('%s is a good time to eat tomatoes' % choice)
else:
    print('%s is a good time to eat asparagus' % choice)

February is a good time to eat oysters


---

If the logic is getting complicated move it into a helper class that determines the condition based on parameters.

In [17]:
def oysters_good(choice):
    choice_lowered = choice.lower()
    return (
        choice_lowered.endswith('r') or
        choice_lowered.endswith('ary'))

def tomatoes_good(choice):
    index = MONTHS.index(choice)
    return 8 > index > 4

In [18]:
time_for_oysters = oysters_good(choice)
time_for_tomatoes = tomatoes_good(choice)

if time_for_oysters:
    print('%s is a good time to eat oysters' % choice)
elif time_for_tomatoes:
    print('%s is a good time to eat tomatoes' % choice)
else:
    print('%s is a good time to eat asparagus' % choice)

February is a good time to eat oysters


---

Alternatively, implement `__nonzero__` or `__bool__` so you can drop the object in place of the old expression. This can reduce the number of delta lines in a refactoring commit, which makes a refactoring feel less scary. It can also be easier to read.

In [27]:
class OystersGood:
    def __init__(self, choice):
        choice = choice
        choice_lowered = choice.lower()
        self.ends_in_r = choice_lowered.endswith('r')
        self.ends_in_ary = choice_lowered.endswith('ary')
        self._result = self.ends_in_r or self.ends_in_ary

    def __bool__(self):  # Equivalent to __nonzero__ in Python 2
        return self._result
            

class TomatoesGood:
    def __init__(self, choice):
        self.index = MONTHS.index(choice)
        self._result = 8 > self.index > 4
    
    def __bool__(self):  # Equivalent to __nonzero__ in Python 2
        return self._result

In [28]:
time_for_oysters = OystersGood(choice)
time_for_tomatoes = TomatoesGood(choice)

if time_for_oysters:
    print('%s is a good time to eat oysters' % choice)
elif time_for_tomatoes:
    print('%s is a good time to eat tomatoes' % choice)
else:
    print('%s is a good time to eat asparagus' % choice)

February is a good time to eat oysters


---

Now the helper function is easy to test and introspect.

In [29]:
test = OystersGood('November')
assert test
assert test.ends_in_r
assert not test.ends_in_ary

test = OystersGood('July')
assert not test
assert not test.ends_in_r
assert not test.ends_in_ary